## Example Use of David's Finance Module, dfinance
#### See it in action!
This notebook shows an example that implements my custom finance package to backtest an (ineffective) algorithmic trading strategy.

In [1]:
import pandas as pd

# won't import if not installed
import dfinance as dfin
from dfinance.trading_strategies import Strategy_SMA_Crossover

In [11]:
# Initialize backtest object
# Initialize portfolio object
my_back = dfin.Backtest()
my_port = dfin.Portfolio()

In [3]:
# Get cash value - can also set this
my_port.cashvalue

100000.0

In [4]:
# # load data from Alpha Vantage csv file
# historical_df = pd.read_csv('gme_minute_data.csv', index_col='time')
# historical_df.index = pd.to_datetime(historical_df.index)
# historical_df.index.name = 'datetime'
# historical_df = historical_df.sort_values(by=['datetime'])

# load data from Yahoo!
data = yf.download(tickers='AAPL', period='60d', progress=False)
data.drop(columns=['Adj Close'])
data.to_csv('aapl_daily_data.csv')
historical_df = pd.read_csv('aapl_daily_data.csv', index_col='Date')

In [5]:
# Process the data from the historical dataframe
my_back.process_historical_data(historical_df,
                                my_port,
                                Strategy_SMA_Crossover.strategy,
                                sma_short=10,
                                sma_long=15,
                                share_num=10,
                                ticker='AAPL')

2021-11-10 | Short: 149.12 | Long: 148.85 | Short > Long

Buying 10 shares of AAPL for 1478.5000610351562

2021-11-11 | Short: 148.91 | Long: 148.84 | Short > Long
2021-11-12 | Short: 149.02 | Long: 148.76 | Short > Long
2021-11-15 | Short: 149.18 | Long: 148.88 | Short > Long
2021-11-16 | Short: 149.25 | Long: 148.91 | Short > Long
2021-11-17 | Short: 149.37 | Long: 149.07 | Short > Long
2021-11-18 | Short: 149.61 | Long: 149.29 | Short > Long
2021-11-19 | Short: 150.26 | Long: 149.97 | Short > Long
2021-11-22 | Short: 151.34 | Long: 150.85 | Short > Long
2021-11-23 | Short: 152.24 | Long: 151.54 | Short > Long
2021-11-24 | Short: 153.42 | Long: 152.20 | Short > Long
2021-11-26 | Short: 154.29 | Long: 152.58 | Short > Long
2021-11-29 | Short: 155.42 | Long: 153.16 | Short > Long
2021-11-30 | Short: 156.47 | Long: 153.81 | Short > Long
2021-12-01 | Short: 157.99 | Long: 154.78 | Short > Long
2021-12-02 | Short: 158.67 | Long: 155.44 | Short > Long
2021-12-03 | Short: 159.33 | Long: 156

In [6]:
my_port.cashvalue

100282.40005493164

In [7]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,AAPL,10,147.850006,1478.500061,buy,2021-11-10
1,AAPL,-10,178.529999,-1785.299988,sell,2021-12-28
2,AAPL,10,177.259995,1772.599945,buy,2021-12-31
3,AAPL,-10,174.820007,-1748.200073,sell,2022-01-12


In [8]:
my_port.summary_of_transactions()

,Ticker,Shares Held,Current Price,Current Value of Shares


In [9]:
# Buy one share of GME for $100 - what a great price!
my_port.buy_stock('GME', 3, 100, '2021-01-01 00:00:00')

In [10]:
my_port.save_transactions('my_port.csv')

In [11]:
my_port.load_transactions('my_port.csv')
my_port.summary_of_transactions()

,Ticker,Shares Held,Current Price,Current Value of Shares
1,GME,3,116.68,350.040001
2,Totals,3,N/A,350.040001


# MultiAsset Short-Only ML Strategy with `dfin`

This section details how an ML driven short-only multi asset strategy can be implemented in `dfin`. 

In [2]:
# hide
# re-imports module changes into notebook on file change
%load_ext autoreload
%autoreload 2

In [56]:
# !pip install lightgbm
from datetime import datetime 

import pandas as pd 

from dfinance.trading_strategies.short_multi_asset_time_series import (
        ShortOnlyMultiAssetStrategy, 
        TimeSeriesModel
        )
from dfinance.utils.ledger import Ledger

ledger = Ledger()

# TODO: clean up loading of this file 
features_df = pd.read_csv(
        "~/Downloads/ETFSample_FibCorrs_MinuteTick_2022_01_15.tsv", 
        sep="\t"
        ).rename(columns = {"Unnamed: 0" : "ts"})
features_df["ts"] = pd.to_datetime(features_df["ts"])
features_df = features_df.set_index("ts")

prices_df = pd.read_csv(
        "~/Downloads/ETFPrices_2008_2022.tsv", 
        sep="\t"
    ).rename(columns = {"Unnamed: 0" : "ts"})
prices_df["ts"] = pd.to_datetime(prices_df["ts"])

prices_df = prices_df.set_index("ts")

strategy = ShortOnlyMultiAssetStrategy(
        features_df = features_df,
        prices_df = prices_df,
        ledger = ledger,
        trade_cost = 0.005,
        tradable_assets = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLK', 'XLU'],
        capital = 5000
)

strategy.test(start_date = '2021-01-01', end_date = '2022-01-10')


257it [00:57,  4.48it/s]


In [77]:
ledger.return_at(
    "2022-01-05", 
    prices_at = prices_df.loc["2022-01-05"].to_dict(orient = "row")[0],
    ).sum()["usd_equity_change"]

/Users/pmk/anaconda3/envs/pmk/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


316.65000000000003

In [79]:
ledger._trans.sort_values("trade_time").head(15)

,ticker,shares,price,trade_time,details
0,XLY,-3,159.42,2021-01-04 05:00:00,{'type': 'sell'}
1,XLP,-7,66.72,2021-01-04 05:00:00,{'type': 'sell'}
2,XLE,-13,37.96,2021-01-04 05:00:00,{'type': 'sell'}
3,XLF,-17,29.08,2021-01-04 05:00:00,{'type': 'sell'}
4,XLV,-4,112.95,2021-01-04 05:00:00,{'type': 'sell'}
5,XLY,3,165.38,2021-01-07 05:00:00,{'type': 'buy'}
6,XLP,7,66.57,2021-01-07 05:00:00,{'type': 'buy'}
7,XLE,13,41.47,2021-01-07 05:00:00,{'type': 'buy'}
8,XLF,17,30.94,2021-01-07 05:00:00,{'type': 'buy'}
9,XLV,4,116.83,2021-01-07 05:00:00,{'type': 'buy'}


In [76]:
ledger.get_balances("2021-01-05")

,ticker,shares,price,trade_time,details,usd_equity_change,current_position_usd_value,net_value_at_timestamp
0,XLY,-3,159.42,2021-01-04 05:00:00,{'type': 'sell'},-478.26,-3,-478.26
1,XLP,-7,66.72,2021-01-04 05:00:00,{'type': 'sell'},-467.04,-7,-467.04
2,XLE,-13,37.96,2021-01-04 05:00:00,{'type': 'sell'},-493.48,-13,-493.48
3,XLF,-17,29.08,2021-01-04 05:00:00,{'type': 'sell'},-494.36,-17,-494.36
4,XLV,-4,112.95,2021-01-04 05:00:00,{'type': 'sell'},-451.80,-4,-451.80


test_data = []